In [25]:
import apache_beam as beam

p = beam.Pipeline()

def Split_Row(input_element):
    return input_element.split(',')

def FilterBasedOnCOuntry(country_name, input_element):
    return input_element[1] == country_name

def CalculateSum(elem):
    return elem[0],(int(elem[2])+int(elem[3])+int(elem[4]))


def FormatText(elem):
    return f'{elem[0]} has received {str(elem[1])} marks'


input_collection = (
    p 
    | beam.io.ReadFromText('source datasets/students_marks.txt') 
    | beam.Map(Split_Row)
)

class MyTransform(beam.PTransform):
  def expand(self, input_col):
    a = (
        input_col
                   | 'Calculate Sum' >> beam.Map(CalculateSum)
                   | 'Apply Formatting' >> beam.Map(FormatText)
                   | 'print data' >> beam.Map(print)
    )
    return a


US_pipeline = (
    input_collection
    | beam.Filter(lambda x : FilterBasedOnCOuntry('US', x))
    # | 'Calculate Sum for US' >> beam.Map(CalculateSum)
    # | 'Apply string formating for US' >> beam.Map(FormatText) 
    # | 'print for US' >> beam.Map(print)
    | "Composite Transformation for US" >> MyTransform()
    | 'Writing result to US file' >> beam.io.WriteToText('output/8/US_Result')
)


IN_pipeline = (
    input_collection
    | beam.Filter(lambda x : FilterBasedOnCOuntry('IN', x))
    # | 'Calculate Sum for India' >> beam.Map(CalculateSum)
    # | 'Apply string formating for India' >> beam.Map(FormatText) 
    #| 'print for IN' >> beam.Map(print)
    | "Composite Transformation for IN" >> MyTransform()
    | 'Writing result to India file' >> beam.io.WriteToText('output/8/IN_Result')
)

p.run()

John has received 82 marks
Jim has received 100 marks
Sam has received 89 marks
Sandhya has received 118 marks
Naina has received 126 marks
